# Hugging Faceで公開されている大規模言語モデルをSageMakerにデプロイ


* 対象モデル
  
  elyza/ELYZA-japanese-Llama-2-7b-fast-instruct
  
  https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct

* HuggingFace Text Generation Inference Containers

  https://huggingface.co/blog/sagemaker-huggingface-llm

  https://aws.amazon.com/jp/blogs/machine-learning/announcing-the-launch-of-new-hugging-face-llm-inference-containers-on-amazon-sagemaker/


### SageMakerライブラリーのインストール

In [1]:
%pip install sagemaker transformers --upgrade


Requirement already up-to-date: sagemaker in /home/vscode/.local/lib/python3.8/site-packages (2.182.0)
     |████████████████████████████████| 7.5 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 17.1 MB/s eta 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0
    Uninstalling transformers-4.26.0:
      Successfully uninstalled transformers-4.26.0
Note: you may need to restart the kernel to use updated packages.


---

### パラメーターを指定

In [2]:
model_id = 'elyza/ELYZA-japanese-Llama-2-7b-fast-instruct'
instance_type = 'ml.g4dn.12xlarge'
gpus = '4'

### インポート

In [3]:
import sagemaker
import boto3


### IAMロールの取得

In [4]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role_name = 'AmazonSageMaker-ExecutionRole-20230617T201891' # Role name with `AmazonSageMakerFullAccess` policy attached
	role = iam.get_role(RoleName=role_name)['Role']['Arn']


Couldn't call 'get_role' to get Role ARN from role name inspiron14 to get Role path.


### SageMakerへデプロイ

In [5]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

image_uri = get_huggingface_llm_image_uri(
  backend='huggingface', # or lmi
  # region=region
)

# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID': model_id, # model_id from hf.co/models
  'HF_TASK':'text-generation',          # NLP task you want to use for predictions
  # 'HF_MODEL_QUANTIZE':'bitsandbytes',
  'SM_NUM_GPUS': gpus,
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
  env=hub,                            # configuration for loading model from Hub
  role=role,                          # IAM role with permissions to create an endpoint
  image_uri=image_uri
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=600,
)


----------!

### 推論

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
text = "クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"

prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)


data = {
   'inputs': prompt,
   'parameters': {
        'max_new_tokens': 256,
        'pad_token_id': tokenizer.pad_token_id,
        'eos_token_id': tokenizer.eos_token_id,
   }
}

# request
result = predictor.predict(data)

result


/home/vscode/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'generated_text': '<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。\n<</SYS>>\n\nクマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。 [/INST] クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を作成します。\n\nクマの名前はキリっとしています。キリっとしたクマは海辺にやってきました。\n\n海辺でキリっとしたクマはアザラシに出会います。\n\nアザラシの名前はフワフワしています。フワフワしたアザラシはキリっとしたクマに話しかけます。\n\n「どうして海辺にいるの？」\n\n「家から遠くにいるのが好きなんです。」\n\n「私は海辺が好きなんです。」\n\n「じゃあ、友達になってください。」\n\n「いいよ。」\n\nキリっとしたクマとフワフワしたアザラシは友達になります。\n\n2匹は海辺でたくさん遊んでいました。\n\n「家に帰りたいです。」\n\n「私も家が好きなんです。」\n\n「じゃあ、家に帰ろう。」\n\nキリっとしたクマとフワフワしたアザラシは家に帰ります。\n\n2匹は家に帰'}]

### エンドポイントの削除

In [7]:
predictor.delete_endpoint(delete_endpoint_config=False)
predictor.delete_model()
